# Band Structure Workflows

This notebook demonstrates how to set up and run band structure calculations
using CrystalMath's band structure workflow system.

You will learn how to:
1. Configure BandStructureWorkflow with BandStructureConfig
2. Use preset k-point paths for common crystal systems
3. Generate band structure inputs from converged SCF
4. Understand the BandStructureResult dataclass
5. Combine convergence studies with band structure

**Prerequisites:**
- `uv sync` in the crystalmath repo
- A converged SCF calculation (or use example)

## 1. Imports

Import the band structure workflow classes and supporting models.

In [ ]:
from crystalmath.workflows.bands import (
    BandStructureWorkflow,
    BandStructureConfig,
    BandStructureResult,
    BandPathPreset,
    HIGH_SYMMETRY_POINTS,
    STANDARD_PATHS,
)
from crystalmath.api import CrystalController
from crystalmath.models import JobSubmission, DftCode, RunnerType

print("Imports successful!")

## 2. Run an SCF Calculation First

Band structure calculations require a converged SCF wavefunction.
Let's submit a simple SCF job first.

In [ ]:
# Initialize controller
ctrl = CrystalController(db_path="bands_demo.db", use_aiida=False)

# Silicon SCF input
scf_input = """Silicon SCF
CRYSTAL
0 0 0
227
5.43
1
14  0.125  0.125  0.125
END
14 3
END
SHRINK
8 8
TOLDEE
8
END
END
"""

# Submit SCF job
scf_job = JobSubmission(
    name="si_scf",
    dft_code=DftCode.CRYSTAL,
    input_content=scf_input,
    runner_type=RunnerType.LOCAL,
)

scf_pk = ctrl.submit_job(scf_job)
print(f"SCF job submitted: PK={scf_pk}")
print("\nNote: In a real workflow, wait for this job to complete before proceeding.")

## 3. Configure Band Structure Workflow

The `BandStructureConfig` dataclass defines all parameters for the workflow.
Key parameters:
- `source_job_pk` - PK of converged SCF job
- `band_path` - K-point path preset (AUTO, FCC, BCC, etc.)
- `kpoints_per_segment` - Resolution along path
- `compute_dos` - Whether to also compute DOS
- `dos_mesh` - K-point mesh for DOS

In [ ]:
# Configure band structure calculation
config = BandStructureConfig(
    source_job_pk=scf_pk,
    band_path=BandPathPreset.FCC,  # Silicon is FCC
    kpoints_per_segment=50,
    compute_dos=True,
    dos_mesh=[12, 12, 12],
    first_band=1,
    last_band=-1,  # All bands
    dft_code="crystal",
    name_prefix="si_bands",
)

print("Band structure workflow configured:")
print(f"  Source job: PK={config.source_job_pk}")
print(f"  Path preset: {config.band_path.value}")
print(f"  K-points/segment: {config.kpoints_per_segment}")
print(f"  Compute DOS: {config.compute_dos}")
print(f"  DOS mesh: {config.dos_mesh}")

## 4. Available K-Path Presets

CrystalMath provides preset k-point paths for common crystal systems.
These follow standard conventions for high-symmetry points.

In [ ]:
print("Available band path presets:\n")

for preset in BandPathPreset:
    print(f"  {preset.value:12s}", end="")
    
    # Show path and points for standard presets
    if preset.value in STANDARD_PATHS:
        path = STANDARD_PATHS[preset.value]
        print(f" -> {'-'.join(path)}")
    else:
        print()

print("\nFor custom paths, use BandPathPreset.CUSTOM")
print("and set config.custom_path = 'G-X-M-G'")

In [ ]:
# Show high-symmetry points for FCC
print("High-symmetry points for FCC structure:\n")

if "fcc" in HIGH_SYMMETRY_POINTS:
    for label, coords in HIGH_SYMMETRY_POINTS["fcc"].items():
        print(f"  {label:8s}: [{coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f}]")

print(f"\nStandard path: {' -> '.join(STANDARD_PATHS['fcc'])}")

## 5. Create Workflow and Generate Inputs

The `BandStructureWorkflow` class generates input files for the calculation.
In a real implementation, you would submit these as jobs.

In [ ]:
# Create workflow instance
workflow = BandStructureWorkflow(config)

print("BandStructureWorkflow created")
print(f"  Config: {config.name_prefix}")
print(f"  Path: {config.band_path.value}")
print("\nThis workflow would generate:")
print("  1. Band structure job (non-SCF along k-path)")
if config.compute_dos:
    print("  2. DOS job (uniform k-mesh)")

## 6. Understanding BandStructureResult

The `BandStructureResult` dataclass contains all results from the workflow.
Let's examine its structure.

In [ ]:
# Create an example result
result = BandStructureResult()

print("BandStructureResult fields:\n")
print(f"  status: {result.status} (pending, running, completed, failed)")
print(f"  band_job_pk: {result.band_job_pk} (PK of band structure job)")
print(f"  dos_job_pk: {result.dos_job_pk} (PK of DOS job if computed)")
print(f"  band_gap_ev: {result.band_gap_ev} (Band gap in eV)")
print(f"  band_gap_type: {result.band_gap_type} (direct or indirect)")
print(f"  is_metal: {result.is_metal} (True if metallic)")
print(f"  fermi_energy_ev: {result.fermi_energy_ev} (Fermi level in eV)")
print(f"  vbm_ev: {result.vbm_ev} (Valence band maximum)")
print(f"  cbm_ev: {result.cbm_ev} (Conduction band minimum)")
print(f"  n_bands: {result.n_bands} (Number of bands computed)")
print(f"  kpath_labels: {result.kpath_labels} (High-symmetry labels)")

In [ ]:
# Example: Simulate a completed result
result_complete = BandStructureResult(
    status="completed",
    band_job_pk=101,
    dos_job_pk=102,
    band_gap_ev=1.12,
    band_gap_type="indirect",
    is_metal=False,
    fermi_energy_ev=5.43,
    vbm_ev=5.43,
    cbm_ev=6.55,
    n_bands=20,
    kpath_labels=["Gamma", "X", "W", "K", "Gamma", "L"],
)

print("Example completed result:")
print(f"  Status: {result_complete.status}")
print(f"  Band gap: {result_complete.band_gap_ev} eV ({result_complete.band_gap_type})")
print(f"  VBM: {result_complete.vbm_ev} eV")
print(f"  CBM: {result_complete.cbm_ev} eV")
print(f"  Number of bands: {result_complete.n_bands}")
print(f"  K-path: {' -> '.join(result_complete.kpath_labels)}")

In [ ]:
# Convert to dict for serialization
result_dict = result_complete.to_dict()

print("Result as dictionary (JSON-serializable):\n")
import json
print(json.dumps(result_dict, indent=2))

## 7. Combining with Convergence Study

Before running band structure, it's recommended to converge k-points first.
Here's how to combine convergence and band structure workflows.

In [ ]:
from crystalmath.workflows.convergence import (
    ConvergenceStudy,
    ConvergenceStudyConfig,
    ConvergenceParameter,
)

# Step 1: Configure convergence study
conv_config = ConvergenceStudyConfig(
    parameter=ConvergenceParameter.SHRINK,
    values=[4, 6, 8, 10, 12, 14],
    base_input=scf_input,
    energy_threshold=0.001,  # 1 meV/atom convergence
    dft_code="crystal",
    name_prefix="si_conv",
)

print("Convergence study configured:")
print(f"  Parameter: {conv_config.parameter.value}")
print(f"  Values: {conv_config.values}")
print(f"  Threshold: {conv_config.energy_threshold} eV/atom")

In [ ]:
# Step 2: Create convergence study and generate inputs
study = ConvergenceStudy(conv_config)
inputs = study.generate_inputs()

print(f"\nGenerated {len(inputs)} convergence jobs:\n")
for name, content_preview in inputs:
    # Show just the name (content would be full input file)
    print(f"  {name}")

print("\nWorkflow:")
print("  1. Submit all convergence jobs")
print("  2. Wait for completion")
print("  3. Analyze results to find converged SHRINK")
print("  4. Use converged parameters for band structure")

## 8. Custom K-Path Example

For specialized structures, you can define custom k-point paths.

In [ ]:
# Custom path configuration
custom_config = BandStructureConfig(
    source_job_pk=scf_pk,
    band_path=BandPathPreset.CUSTOM,
    custom_path="Gamma-X-M-Gamma-R-X",  # Custom path string
    kpoints_per_segment=40,
    compute_dos=False,
    dft_code="crystal",
    name_prefix="custom_bands",
)

print("Custom band path configuration:")
print(f"  Path: {custom_config.custom_path}")
print(f"  Resolution: {custom_config.kpoints_per_segment} pts/segment")
print("\nNote: Custom paths require you to define the high-symmetry")
print("      point coordinates in fractional reciprocal space.")

## Summary

In this notebook, you learned:

1. **Configure** band structure workflows with `BandStructureConfig`
2. **Use** preset k-point paths for common crystal systems
3. **Understand** high-symmetry points and paths
4. **Interpret** `BandStructureResult` dataclass fields
5. **Combine** convergence studies with band structure
6. **Create** custom k-point paths for specialized structures

**Key Classes:**
- `BandStructureConfig` - Workflow configuration
- `BandStructureWorkflow` - Workflow orchestration
- `BandStructureResult` - Results container
- `BandPathPreset` - Preset k-paths (AUTO, FCC, BCC, etc.)

**Preset Paths:**
- `AUTO` - Auto-detect from structure symmetry
- `FCC` - Face-centered cubic (Gamma-X-W-K-Gamma-L-U-W-L-K)
- `BCC` - Body-centered cubic (Gamma-H-N-Gamma-P-H)
- `CUBIC` - Simple cubic (Gamma-X-M-Gamma-R-X)
- `HEXAGONAL` - Hexagonal (Gamma-M-K-Gamma-A-L-H-A)
- `TETRAGONAL` - Tetragonal (Gamma-X-M-Gamma-Z-R-A-Z)
- `CUSTOM` - User-defined path

**Recommended Workflow:**
1. Run convergence study for k-points
2. Run SCF with converged parameters
3. Run band structure from converged SCF
4. Optionally compute DOS on dense mesh

**Next Steps:**
- `04_convergence_eos.ipynb` - Convergence studies and equation of state
- `05_templates_workflows.ipynb` - Template system and phonon workflows